In [0]:
!pip install numba==0.53

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-f54c86bf-3ef5-419b-b21a-c430964622bf/bin/python -m pip install --upgrade pip' command.


In [0]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from mlflow.tracking import MlflowClient
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima.stationarity import ADFTest
from sklearn.model_selection import ParameterGrid
from matplotlib.pyplot import figure
from mlflow import pyfunc
from pyspark.sql import SQLContext

import sys
import os
import glob
import datetime
import mlflow
import random
import seaborn as sns
import os.path
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import pmdarima as pm
import joblib
import requests
import json

from scipy import stats
from pandas import Series
from scipy.special import boxcox, inv_boxcox
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

# From Spiros
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
import matplotlib as mpl
#import seaborn as sns
import pickle
#import glob

import scipy.cluster.hierarchy as shc
from scipy import stats
from scipy.special import boxcox, inv_boxcox
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_samples, silhouette_score, davies_bouldin_score
from sklearn.neighbors import NearestNeighbors
from random import sample
import random
from numpy.random import uniform
from math import isnan
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
from kshape.core import kshape
from tslearn.clustering import KShape, TimeSeriesKMeans
from sklearn.model_selection import RandomizedSearchCV
from numpy import array, newaxis
from numpy import hstack
from math import sqrt 

import gc
import time

# Use Keras with Python 3.5
import keras
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, GRU, Input, Concatenate, Reshape, Lambda, Bidirectional
# from keras.layers.normalization import BatchNormalization # tf.keras.layers.BatchNormalization()
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.models import load_model
# from tensorflow import set_random_seed # set_random_seed(x) --> tensorflow.random.set_seed(x)
import joblib # from sklearn.externals import joblib
from sklearn.decomposition import PCA
import shap

import itertools



import statsmodels.api as sm
#from scipy import stats

#-------------------------- for time series clustering ----------------------------
from tslearn.metrics import dtw
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
#----------------------------------------------------------------------------------


#-------------------------- for non time series clustering ----------------------------
from sklearn.metrics.pairwise import euclidean_distances
#---------------------------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler


# We will use the following indicators for evaluating the model’s performance
from statsmodels.tools.eval_measures import rmse

#-------------------------- for Seep learning (RNN / MLP) ----------------------------
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
tf.random.set_seed(1234)

In [0]:
"""#==================================================================================================="""
"""#============================================== Functions =========================================="""
"""#==================================================================================================="""
def import_json(json_path):
    df = pd.read_json(json_path)
    normalized_data = pd.json_normalize(df.iloc[0,0])
    normalized_data['time'] = pd.to_datetime(normalized_data['time'], utc=True)
    normalized_data = normalized_data.set_index('time')
    return normalized_data

#================== Evaluating the model ================
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    #mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    #return map
    sum_ae = 0.0
    mean_Yactual = np.mean(Y_actual)
    for jj in range(len(Y_actual)):
        if Y_actual[jj]==0:
            sum_ae = sum_ae + np.abs((Y_actual[jj] - Y_Predicted[jj])/mean_Yactual)
        else:
            sum_ae = sum_ae + np.abs((Y_actual[jj] - Y_Predicted[jj])/Y_actual[jj])
        mape = (sum_ae/len(Y_actual))*100
    return mape

def add_calendar_components(data, calendar_components=None, drop_constant_columns=True):
    """
    Add calendar components year, quarter, month, weekday, dayofyear, hour
    to the input DataFrame.

    :param data: input DataFrame with a DateTimeIndex and at least one column.
    :param calendar_components: List of strings, specifying the calendar components you want to add in
        ["year", "quarter", "month", "week", "day", "hour"].
    :param drop_constant_columns: If True, drops constant calendar components.
    :return: new DataFrame with the added calendar components.
    """

    if data.empty or not isinstance(data, pd.DataFrame) or not isinstance(data.index, pd.DatetimeIndex):
        raise ValueError("Input must be a non-empty pandas DataFrame with a DateTimeIndex.")

    default_components = ["year", "quarter", "month", "weekday", "dayofyear", "hour"]

    if calendar_components is not None:
        if not set(calendar_components).issubset(default_components):
            raise ValueError("Argument 'calendar_components' must be a subset of "
                             "['year', 'quarter', 'month', 'weekday', 'dayofyear', 'hour'].")
    else:
        calendar_components = default_components

    df_new = data.assign(**{
        '{}'.format(component): getattr(data.index, component)
        for component in calendar_components
    })

    if drop_constant_columns:
        df_new = df_new.loc[:, (df_new[calendar_components] != df_new.iloc[0]).any()]

    return df_new

In [0]:
#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(df):
    result=adfuller(df)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [0]:
def run_mlflow(model, name, exp_id):

    with mlflow.start_run(run_name=name) as run:

        mlflow.autolog(log_models=True) 
        mlflow.log_param("order", my_score )
        mlflow.log_param("seasonal_order", my_sea_score )
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2", r2)
        mlflow.log_artifact("sarimax.png")
        run_id = run.info.run_id
        path = "dbfs:/databricks/mlflow-tracking/{exp_id}/{run_id}".format(exp_id = exp_id, run_id = run_id)
        mlflow.statsmodels.save_model(model, path)
        print("Run ID: " + run_id)
        print("run completed!")

In [0]:
'''
Evaluation metrics for predictions.
:parameter
    :param dtf: DataFrame with columns raw values, fitted training  
                 values, predicted test values
:return
    dataframe with raw ts and forecast
'''
def evaluate_forecast(dtf, title, plot=True):
    try:
        ## residuals
        dtf["error"] = dtf["ts"] - dtf["forecast"]
        dtf["error_pct"] = dtf["error"] / dtf["ts"]
        
        ## kpi
        error_mean = dtf["error"].mean()
        error_std = dtf["error"].std()
        #mape = dtf["error_pct"].apply(lambda x: np.abs(x)).mean()
        global mae, mse, rmse, r2
        mae = dtf["error"].apply(lambda x: np.abs(x)).mean()
        mse = dtf["error"].apply(lambda x: x**2).mean()
        rmse = np.sqrt(mse)  #root mean squared error
        if np.mean(dtf["ts"]) in dtf["ts"].values:   
          r2= 1-(sum(dtf["error"]**2)/sum((dtf["ts"]-(np.mean(dtf["ts"] - 0.01))**2)))
        else: 
          r2= 1-(sum(dtf["error"]**2)/sum((dtf["ts"]-np.mean(dtf["ts"]))**2))
        
        ## intervals
        dtf["pred_int_low"] = dtf["forecast"] - 1.96*error_std
        dtf["pred_int_up"] = dtf["forecast"] + 1.96*error_std
        
        ## plot
        if plot==True:
            dtf = dtf.reset_index()
            dtf['time'] = pd.to_datetime(dtf['time'])
            dtf = dtf.set_index('time')
            dtf = dtf.resample('1H').mean().fillna(0) 
            dtf[['ts','forecast']].plot(figsize=(12,8))
            plt.xlabel('Date') 
            plt.ylabel('Consomation (Wh)')  
            plt.title("Power value")
            plt.legend()
            plt.savefig("sarimax.png")
           
        print("Test --> Error mean:", np.round(error_mean), " | r2:", np.round(r2,2),
                  " | mae:",np.round(mae), " | rmse:",np.round(rmse))
        
        return dtf, mae, rmse, r2
      
    
    except Exception as e:
        print("--- got error ---")
        print(e)

In [0]:
def test_stationarity_acf_pacf(data, sample, max_lag):
    
    """
    This function tests the stationarity and plot the autocorrelation
    and partial autocorrelation of the time series.
    Test stationarity by:
    - running Augmented Dickey-Fuller test with 95%
    In statistics, the Dickey–Fuller test tests the null hypothesis
    that a unit root is present in an autoregressive model.
    The alternative hypothesis is different depending
    on which version of the test is used,
    but is usually stationarity or trend-stationarity.
    - plotting mean and variance of a sample from data
    - plotting autocorrelation and partial autocorrelation

    p-value > 0.05: Fail to reject the null hypothesis (H0),
    the data has a unit root and is non-stationary.
    p-value <= 0.05: Reject the null hypothesis (H0),
    the data does not have a unit root and is stationary.
    This function is used to verify stationarity
    so that suitable forecasting methods can be applied.

    Partial autocorrelation is a summary of the relationship
    between an observation in a time series with observations
    at prior time steps with the relationships of intervening
    observations removed. The partial autocorrelation at lag k
    is the correlation that results after removing the effect
    of any correlations due to the terms at shorter lags.

    The autocorrelation for an observation and an observation
    at a prior time step comprises both the direct correlation
    and indirect correlations. These indirect correlations are
    a linear function of the correlation of the observation,
    with observations at intervening time steps.

    These correlations are used to define the parameters
    of the forecasting methods (lag).

    :parameter
     :param data: timeSeries for which the stationarity as to be evaluated.
     :param sample: Sample (float) of the data that will be evaluated.
     :param max_lag: Maximum number of lag which included in test.
                    The default value is 12*(nobs/100)^{1/4}.
    :return:
      plot of the mean and variance of the sample with the p-value
      and plot of the autocorrelation and partial autocorrelation.
    """
    
    if data.empty:
        raise ValueError("Input series must be not empty.")

    elif 0.0 > sample or sample > 1.0:
        raise ValueError("Sample value should be between 0 and 1")

    for col in data:
      ts_ax = plt.subplot2grid(shape=(2, 2), loc=(0, 0), colspan=2)
      pacf_ax = plt.subplot2grid(shape=(2, 2), loc=(1, 0))
      acf_ax = plt.subplot2grid(shape=(2, 2), loc=(1, 1))
      dtf_ts = data[col].to_frame(name="ts")
      sample_size = int(len(data) * sample)
      dtf_ts["mean"] = dtf_ts["ts"].head(sample_size).mean()
      dtf_ts["lower"] = dtf_ts["ts"].head(sample_size).mean() + dtf_ts["ts"].head(sample_size).std()
      dtf_ts["upper"] = dtf_ts["ts"].head(sample_size).mean() - dtf_ts["ts"].head(sample_size).std()
      dtf_ts["ts"].plot(ax=ts_ax, color="black", legend=False)
      dtf_ts["mean"].plot(
          ax=ts_ax, legend=False, color="red",
          linestyle="--", linewidth=0.7)
      ts_ax.fill_between(
          x=dtf_ts.index, y1=dtf_ts['lower'],
          y2=dtf_ts['upper'], color='lightskyblue', alpha=0.4)
      dtf_ts["mean"].head(sample_size).plot(
          ax=ts_ax, legend=False,
          color="red", linewidth=0.9)
      ts_ax.fill_between(
          x=dtf_ts.index, y1=dtf_ts['lower'],
          y2=dtf_ts['upper'], color='lightskyblue', alpha=0.4)
      dtf_ts["mean"].head(sample_size).plot(
          ax=ts_ax, legend=False,
          color="red", linewidth=0.9)
      ts_ax.fill_between(
          x=dtf_ts.head(sample_size).index,
          y1=dtf_ts['lower'].head(sample_size),
          y2=dtf_ts['upper'].head(sample_size), color='lightskyblue')
      adfuller_test = sm.tsa.stattools.adfuller(
          data, maxlag=max_lag, autolag="AIC")
      adf, p, critical_value = adfuller_test[0], adfuller_test[1], adfuller_test[4]["5%"]
      p = round(p, 3)
      conclusion = "Stationary" if p < 0.05 else "Non-Stationary"
      ts_ax.set_title(
          'Dickey-Fuller Test 95%: ' + conclusion +
          '(p value: ' + str(p) + ')')

      # pacf (for AR) and acf (for MA)
      smt.graphics.plot_pacf(
          data, lags=30, ax=pacf_ax,
          title="Partial Autocorrelation (for AR component)")
      smt.graphics.plot_acf(
          data, lags=30, ax=acf_ax,
          title="Autocorrelation (for MA component)")
      plt.tight_layout()

In [0]:
def test_sarimax(ts_train, ts_test, exog_test, p, model):
    """
    The function uses the model from the fit_sarimax function
    to make predictions for the future value.

    :parameter
      :param ts_train: timeSeries used to train the model.
      :param ts_test: timeSeries used to test the model.
      :param exog_test: timeSeries containing the exogeneous variables.
      :param p: number of periods to be forcasted.
      :param model: model from the fit_sarimax function.

    :return
      Dataframe containing the true values and the forecasted ones.
    """

    if ts_train.empty:
        raise ValueError("Train series must be not empty.")
    elif ts_test.empty:
        raise ValueError("Test series must be not empty.")
    elif not isinstance(p, int):
        raise ValueError("p must be an integer.")

    for col in range(ts_test.shape[1]):
      dtf_test = ts_test.iloc[:p, col].to_frame(name="ts")

    if exog_test is None:
        dtf_test["forecast"] = model.predict(
            start=len(ts_train) ,
            end=len(ts_train) + len(ts_test[1:p]),
            exog=exog_test)
    else:
        dtf_test["forecast"] = model.predict(
            start=len(ts_train) ,
            end=len(ts_train) + len(ts_test[:p-1]),
            exog=exog_test[:p])
    dtf_test = dtf_test.round()
    return dtf_test

In [0]:
def fit_sarimax(ts_train, order, seasonal_order, exog_train=None):
    """
    Fit SARIMAX (Seasonal ARIMA with External Regressors):
    y[t+1] = (c + a0*y[t] + a1*y[t-1] +...+ ap*y[t-p]) + (e[t] +
                  b1*e[t-1] + b2*e[t-2] +...+ bq*e[t-q]) + (B*X[t])
    :parameter
    :param ts_train: pandas timeseries
    :param order: tuple - ARIMA(p,d,q) --> p: lag order (AR), d:
    degree of differencing (to remove trend), q: order
                        of moving average (MA).
    :param seasonal_order: tuple - (P,D,Q,s) --> s: number of
                        observations per seasonal (ex. 7 for weekly
                        seasonality with daily data, 12 for yearly
                        seasonality with monthly data).
    :param exog_train: pandas dataframe or numpy array.

    :return
    Model and dtf with the fitted values
    """
    # train
    if ts_train.empty:
        raise ValueError("Train series must be not empty.")

    model = smt.SARIMAX(
        ts_train, order=order, seasonal_order=seasonal_order,
        exog=exog_train, enforce_stationarity=False,
        enforce_invertibility=False, freq='H')
    model = model.fit()
    for col in ts_train:
        dtf_train = ts_train[col].to_frame(name="ts")
        dtf_train["model"] = model.fittedvalues

    return dtf_train, model

In [0]:
def param_tuning_sarimax(data, m, max_order, information_criterion='aic'):
    """
    Automatically discover the optimal order for a SARIMAX model.
    The function works by conducting differencing tests to determine
    the order of differencing, d, and then fitting models within ranges
    of defined start_p, max_p, start_q, max_q ranges.

    If the seasonal optional is enabled(allowing SARIMAX over ARIMA),
    it also seeks to identify the optimal P and Q hyperparameters
    after conducting the Canova-Hansen
    to determine the optimal order of seasonal differencing, D.

    In order to find the best model, it optimizes for
    a given information_criterion
    and returns the ARIMA which minimizes the value.

    :parameter
      :param data: timeSeries used to fit the sarimax estimator.
      :param m: refers to the number of periods in each season.
                      For example, m is 4 for quarterly data,
                      12 for monthly data, or 1 for annual data.
      :param max_order: maximum value of p+q+P+Q.
                      If p+q >= max_order, a model will not be
                      fit with those parameters and will progress
                      to the next combination. Default is 5.
      :param information_criterion: used to select the best model.
                    Possibilities are ‘aic’, ‘bic’, ‘hqic’, ‘oob’.
                    Default is 'aic'.

    :return
      best_model: model with the optimal parameters
    """
    if data.empty:
        raise ValueError("Input series must be not empty.")
    elif not isinstance(m, int):
        raise ValueError("m must be an integer.")

    best_model = pm.auto_arima(data, start_p=1, start_q=1, start_P=2, start_Q=1,
                               test='adf',  # use adftest to find optimal 'd'
                               max_p=max_order, max_q=max_order,  # maximum p and q
                               max_P=max_order, max_D=1,  # maximum P and D and Q
                               max_Q=max_order,
                               m=m,  # frequency of series # m=12 Monthly m=24 Hourly
                               d=None,  # let model determine 'd'
                               seasonal=True,  
                               trace=True,
                               exogenous=None,
                               error_action='ignore',
                               suppress_warnings=True,
                               stepwise=True)

    return best_model

In [0]:
def split_train_test(ts, test=0.1, plot=True, figsize=(15,5)):
    ## define splitting point
    if type(test) is float:
        split = int(len(ts)*(1-test))
        perc = test
    elif type(test) is str:
        split = ts.reset_index()[ 
                      ts.reset_index().iloc[:,0]==test].index[0]
        perc = round(len(ts[split:])/len(ts), 2)
    else:
        split = test
        perc = round(len(ts[split:])/len(ts), 2)
    print("--- splitting at index: ", split, "|", 
          ts.index[split], "| test size:", perc, " ---")
    
    ## split ts
    global ts_train
    global ts_test
    ts_train = ts.head(split)
    ts_test = ts.tail(len(ts)-split)

    if plot is True:
        fig, ax = plt.subplots(nrows=1, ncols=2, sharex=False, 
                               sharey=True, figsize=figsize)
        ts_train.plot(ax=ax[0], grid=True, title="Train", 
                      color="black")
        ts_test.plot(ax=ax[1], grid=True, title="Test", 
                     color="blue")
        ax[0].set(xlabel=None)
        ax[1].set(xlabel=None)
        plt.show()

        return ts_train, ts_test